In [1]:
import cv2
from hand_detector import HandDetector
from visualizer import AnnotationsVisualizer
from gesture_classifier import ThumbDetector, MockClassifier
from game_logic.game_controller import GameController

In [2]:
mirror_display = True

cap = cv2.VideoCapture(0)

visualizer = AnnotationsVisualizer()
thumb_detector = ThumbDetector()
move_classifier = MockClassifier()
game_controller = GameController(thumb_detector, move_classifier)

try:
    with HandDetector(
        user_perspective=True,
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7   
    ) as detector:
        
        print("Gra uruchomiona! Pokaż kciuk w górę aby zacząć.")
        print("Naciśnij ESC aby wyjść.")
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # wykrywanie łapy
            detected_hands = detector.detect(frame)
            
            # aktualizacja stanu gry
            game_controller.update(detected_hands)
            
            # obrot obrazem 
            frame_to_show = cv2.flip(frame, 1) if mirror_display else frame
            
            # nakladanie landmarkow dłoni
            frame_to_show = visualizer.render(
                frame_to_show, detected_hands, mirror_display)
            
            # UI gry
            frame_to_show = game_controller.render_ui(frame_to_show)
            
            # wyswietlanie
            cv2.imshow("Rock Paper Scissors AI Game", frame_to_show)
            
            # Wyjście ESC lub po zakończeniu gry
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC
                break
            
            # reset gry przez 'R'
            if key == ord('r') or key == ord('R'):
                game_controller.reset()
                print("Gra zresetowana!")
                
finally:
    cap.release()
    cv2.destroyAllWindows()
    print("\nGra zakończona!")
    print(f"Wynik końcowy: Gracz {game_controller.player_score} - {game_controller.computer_score} Komputer")

Gra uruchomiona! Pokaż kciuk w górę aby zacząć.
Naciśnij ESC aby wyjść.


c:\Users\c15ab\Desktop\SinuzRPS\rock-paper-scissors-ai-game\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Gra zresetowana!

Gra zakończona!
Wynik końcowy: Gracz 0 - 0 Komputer


In [10]:
# wyswietl historię wszystkich rund
print("\n=== HISTORIA GRY ===")
for record in game_controller.match_history:
    print(f"Runda {record.round_number}: "
          f"Gracz: {record.player_move.name}, "
          f"Komputer: {record.computer_move.name}, "
          f"Wynik: {record.outcome.name}")


=== HISTORIA GRY ===
